In [ ]:
import json
from time import time
from itertools import combinations
from operator import itemgetter
from tqdm import tqdm
import numpy as np
from scipy.spatial import Delaunay
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
COORDINATE_THRESHOLD = 19.2 / 69  # Distance threshold for edge existence
DISTANCE_FACTOR = 100000  # Factor to multiply distance by to get integer weights
MAX_COORDINATE_VALUE = int(COORDINATE_THRESHOLD * DISTANCE_FACTOR)
N_VERTEX = 2716 # found from a previous run of len(g.vs)


file_geo = "./data/los_angeles_censustracts.json"
file_time = "./data/los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv"


lat_long = np.empty((N_VERTEX, 2), dtype=float)
with open(file_geo, 'r') as f:
    d = json.load(f)
    features = d['features']

    for feature in features:
        geometry_type = feature['geometry']['type']
        centroid = np.array(feature['geometry']['coordinates'][0]) if geometry_type == 'Polygon' else np.array(
            feature['geometry']['coordinates'][0][0])

        movement_id = feature['properties']['MOVEMENT_ID']
        lat_long[int(movement_id) - 1] = centroid.mean(axis=0)[::-1]

delaunay_out = Delaunay(lat_long)
edges = delaunay_out.simplices


g_temp = ig.Graph()
g_temp.add_vertices(len(delaunay_out.points))

g_temp.vs['lat'] = lat_long[:, 0]
g_temp.vs['lon'] = lat_long[:, 1]

# Add edges to the graph based on coordinate threshold
for simplex in delaunay_out.simplices:
    edges = [(simplex[0], simplex[1]), (simplex[0], simplex[2]), (simplex[1], simplex[2])]
    for edge in edges:
        edge = tuple(sorted(edge))
        v1 = g_temp.vs[edge[0]]
        v2 = g_temp.vs[edge[1]]
        v1_lat, v1_lon = v1['lat'], v1['lon']
        v2_lat, v2_lon = v2['lat'], v2['lon']
        distance = np.sqrt((v1_lat - v2_lat) ** 2 + (v1_lon - v2_lon) ** 2)
        if distance < COORDINATE_THRESHOLD:
            g_temp.add_edge(edge[0], edge[1], weight=distance)

In [ ]:
from queue import PriorityQueue

class Dijkstra:
    
    DISTANCE_FACTOR = 100000  # Factor to multiply distance by to get integer weights

    def __init__(self, g: ig.Graph, n_vertices: int):
        dtype = np.int32
        self._n_vertices = n_vertices
        self._dist = np.empty(n_vertices, dtype=dtype)
        self._q = PriorityQueue()
        self.dist_matrix = np.ones((n_vertices, n_vertices), dtype=dtype) * 2 * MAX_COORDINATE_VALUE
        
        # Create adjacency matrix
        self._adj = np.ones((n_vertices, n_vertices), dtype=dtype) * 2 * MAX_COORDINATE_VALUE
        self._neigbhors = [[] for _ in range(n_vertices)]
        for u in range(n_vertices):
            for v in g.neighbors(u):
                self._neigbhors[u].append(v)
                val = int(g.es[g.get_eid(u, v)]['weight']*DISTANCE_FACTOR)
                self._adj[u, v] = val
                self._adj[v, u] = val

    def run(self, source: int) -> np.ndarray:
        """Run Dijkstra's algorithm on a graph

        Parameters
        ----------
        source: int
            vertex node (0-indexed)

        Returns
        -------
        shortest_distances: np.ndarray
            Array of shortest distances from source to all other vertices
        """
        # Initialize
        self._dist.fill(2 * MAX_COORDINATE_VALUE)
        self._dist[source] = 0
        for v in range(self._n_vertices):
            self._q.put((self._dist[v], v))

        # Run Dijkstra
        while not self._q.empty():
            _, u = self._q.get()

            for v in self._neigbhors[u]:
                alt = self._dist[u] + self._adj[u, v]
                if alt < self._dist[v]:
                    self.dist_matrix[source, v] = alt
                    self.dist_matrix[v, source] = alt
                    self._dist[v] = alt
                    self._q.put((alt, v))

        self._dist[source] = 2 * MAX_COORDINATE_VALUE
        return self._dist / DISTANCE_FACTOR
    

dijkstra = Dijkstra(g_temp, n_vertices=N_VERTEX)

In [ ]:
distances = np.empty((N_VERTEX, N_VERTEX), dtype=np.float32)
for i in tqdm(range(N_VERTEX)):
    distances[i] = dijkstra.run(source=i)

In [ ]:
# Calculate extra distance between all pairs of points
extra_distances = []
vertices = g_temp.vs()
for v1, v2 in combinations(vertices, 2):
    dijkstra_dist = distances[v1.index, v2.index]
    euclidean_distance = np.sqrt((v1['lat'] - v2['lat']) ** 2 + (v1['lon'] - v2['lon']) ** 2)
    euclidean_distance = euclidean_distance
    extra_distance = dijkstra_dist - euclidean_distance
    extra_distances.append((v1.index, v2.index, extra_distance))

In [ ]:
# Get the top 20 pairs with the highest extra distance
extra_distances.sort(key=itemgetter(2), reverse=True)
top_pairs = extra_distances[:20]

# Print the source and destination of the top pairs
for pair in top_pairs:
    source = vertices[pair[0]]
    destination = vertices[pair[1]]
    # print(f"Source: ({source['lat']}, {source['lon']}), Destination: ({destination['lat']}, {destination['lon']})")

# Create new edges in the graph for the top pairs
for pair in top_pairs:
    source_index = pair[0]
    destination_index = pair[1]
    g_temp.add_edge(source_index, destination_index)

# Plot the graph
lon_coords = g_temp.vs['lon']
lat_coords = g_temp.vs['lat']
new_edges = []
fig = go.Figure()
for i, edge in enumerate(top_pairs):
    x1, y1 = lon_coords[edge[0]], lat_coords[edge[0]]
    x2, y2 = lon_coords[edge[1]], lat_coords[edge[1]]
    fig.add_trace(go.Scattermapbox(
        mode = "lines",
        lon = [x1, x2],
        lat = [y1, y2],
        marker = {'size': 10}
    ))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -118.2437, 'lat': 34.0522},
        'style': "stamen-terrain",
        'zoom': 8}
)

fig.show()